In [ ]:
#Import basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import os 

# Load in excel file and define future save directory
indonesia = pd.ExcelFile(r"D:\Projects\TMU\Indonesia_Dengue\DENGUE MONTHLY DATA.xlsx")
save_dir = r"D:\Projects\TMU\Indonesia_Dengue"

Set variables and clean up data.

In [95]:
def rename_excel_sheets(excel_path):
    '''Function to load in whole excel sheet, load in as individual dataframes, and rename columns.'''
    excel_file = pd.ExcelFile(excel_path)
    sheet_names = excel_file.sheet_names
    
    if len(sheet_names) > 14:
        raise ValueError("Excel file has more than 14 sheets, cannot map to 2010-2023 range.")
    
    new_names = [f"in_{year}" for year in range(2010, 2010 + len(sheet_names))]
    
    dataframes = {new_name: excel_file.parse(sheet_name=old_name) for old_name, new_name in zip(sheet_names, new_names)}
    
    rename_dict = {3: 'Infection_1', 4: 'Death_1',
                   5: 'Infection_2', 6: 'Death_2',
                   7: 'Infection_3', 8: 'Death_3',
                   9: 'Infection_4', 10: 'Death_4',
                   11: 'Infection_5', 12: 'Death_5',
                   13: 'Infection_6', 14: 'Death_6',
                   15: 'Infection_7', 16: 'Death_7',
                   17: 'Infection_8', 18: 'Death_8',
                   19: 'Infection_9', 20: 'Death_9',
                   21: 'Infection_10', 22: 'Death_10',
                   23: 'Infection_11', 24: 'Death_11',
                   25: 'Infection_12', 26: 'Death_12'}
    
    for name, df in dataframes.items():
        # Rename columns
        df.columns = [rename_dict[i] if i in rename_dict else col for i, col in enumerate(df.columns)]
        
        # Drop the first row (index 0)
        df.drop(index=0, inplace=True)
        
        # Create a global variable with the name corresponding to the sheet
        globals()[name] = df

    # Optionally, return None, as DataFrame instances are now created in the global scope


In [101]:
rename_excel_sheets(indonesia)

Functions to plot yearly, multi-yearly trend curves.

In [97]:
def plot_infection_trend(df, year, save_dir):
    """
    Summarize monthly infection data and save the plot for a given year.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing infection data for a specific year.
    - year (int): The year corresponding to the dataframe.
    - save_dir (str): The directory to save the plot image.
    
    The dataframe must contain columns 'Infection_1' to 'Infection_12', 
    each representing infection counts for each month from January to December.
    """
    # List of columns representing infection counts from January to December
    infection_columns = ['Infection_1', 'Infection_2', 'Infection_3', 'Infection_4', 
                         'Infection_5', 'Infection_6', 'Infection_7', 'Infection_8', 
                         'Infection_9', 'Infection_10', 'Infection_11', 'Infection_12']

    # Check if the infection columns exist in the dataframe
    if not all(col in df.columns for col in infection_columns):
        print(f"One or more infection columns are missing in {year} data.")
        return

    # Exclude the first row (index 0) as it is null and we only want data from states
    df_clean = df[infection_columns].iloc[1:]  # Exclude first row

    # Convert all columns to numeric (to avoid issues with non-numeric values)
    df_clean = df_clean.apply(pd.to_numeric, errors='coerce')

    # Handle missing values: Filling missing values with 0 (or another strategy)
    df_clean = df_clean.fillna(0)

    # Sum the infection data across all rows (states)
    monthly_totals = df_clean.sum(axis=0)

    # Ensure the monthly totals have exactly 12 values
    if len(monthly_totals) == 12:
        # Create a list of month names
        months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
                  'September', 'October', 'November', 'December']

        # Plot the curve
        plt.figure(figsize=(10, 6))
        plt.plot(months, monthly_totals, marker='o', linestyle='-', color='b', label=f'New Infections ({year})')
        plt.title(f'Monthly Trend of New Infections in Indonesia ({year})', fontsize=14)
        plt.xlabel('Month', fontsize=12)
        plt.ylabel('Total New Infections', fontsize=12)
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()

        # Save the plot as a file in the specified directory
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)  # Create directory if it doesn't exist
        save_path = os.path.join(save_dir, f'infection_trend_{year}.png')
        plt.savefig(save_path)

        # Close the plot to free memory
        plt.close()
        print(f"Plot for {year} saved at {save_path}")
    else:
        print(f"The data structure is incorrect for {year} data. Please check the infection columns.")

# For loop to plot annual plots for 2010 to 2023
for year in range(2010, 2024):  # Loop from 2010 to 2023
    in_data = globals().get(f'in_{year}')  # Dynamically access in_2010, in_2011, ..., in_2023
    plot_infection_trend(in_data, year, save_dir)


Plot for 2010 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2010.png
Plot for 2011 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2011.png
Plot for 2012 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2012.png
Plot for 2013 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2013.png
Plot for 2014 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2014.png
Plot for 2015 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2015.png
Plot for 2016 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2016.png
Plot for 2017 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2017.png
Plot for 2018 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2018.png
Plot for 2019 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2019.png
Plot for 2020 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2020.png
Plot for 2021 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2021.png
Plot for 2022 saved at D:\Pr

In [98]:
def plot_combined_infection_trend(df_dict, save_dir):
    """
    Summarize monthly infection data from 2010 to 2023 and save a single plot.
    
    Parameters:
    - df_dict (dict): A dictionary where keys are years and values are dataframes
      containing infection data for each year.
    - save_dir (str): The directory to save the plot image.
    
    Each dataframe must contain columns 'Infection_1' to 'Infection_12', 
    each representing infection counts for each month from January to December.
    """
    # List of columns representing infection counts from January to December
    infection_columns = ['Infection_1', 'Infection_2', 'Infection_3', 'Infection_4', 
                         'Infection_5', 'Infection_6', 'Infection_7', 'Infection_8', 
                         'Infection_9', 'Infection_10', 'Infection_11', 'Infection_12']

    # Initialize a list to collect monthly infection totals for all years
    all_months = []
    all_infection_totals = []

    # Initialize a list to mark the year positions on the x-axis
    year_positions = []

    # Iterate over the years and process the infection data
    for year, df in df_dict.items():
        # Check if the infection columns exist in the dataframe
        if not all(col in df.columns for col in infection_columns):
            print(f"One or more infection columns are missing in {year} data.")
            continue

        # Exclude the first row (index 0) as it is null and we only want data from states
        df_clean = df[infection_columns].iloc[1:]  # Exclude first row

        # Convert all columns to numeric (to avoid issues with non-numeric values)
        df_clean = df_clean.apply(pd.to_numeric, errors='coerce')

        # Handle missing values: Filling missing values with 0 (or another strategy)
        df_clean = df_clean.fillna(0)

        # Sum the infection data across all rows (states)
        monthly_totals = df_clean.sum(axis=0)

        # Append the months and corresponding totals to the lists
        months = [f"{month} {year}" for month in ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
                                                  'September', 'October', 'November', 'December']]
        all_months.extend(months)
        all_infection_totals.extend(monthly_totals)

        # Record the positions of the year labels for marking the year boundaries
        year_positions.append(len(all_months))  # This will correspond to the last month of each year

    # Plot the combined infection trend across all years
    plt.figure(figsize=(12, 6))
    plt.plot(all_months, all_infection_totals, marker='o', linestyle='-', color='b', label='Total New Infections (2010-2023)')

    # Add vertical red lines for each year boundary
    for year_pos in year_positions:
        plt.axvline(x=year_pos - 0.5, color='r', linestyle='--', lw=1)

    # Set the title and labels
    plt.title('Monthly Trend of New Infections in Indonesia (2010-2023)', fontsize=14)
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Total New Infections', fontsize=12)

    # Keep only the year labels on the x-axis
    year_labels = [f"{year}" for year in range(2010, 2024)]
    plt.xticks(year_positions, year_labels, rotation=45)  # Position the labels at the end of each year

    # Add grid and adjust layout
    plt.grid(True)
    plt.tight_layout()

    # Save the plot as a file in the specified directory
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)  # Create directory if it doesn't exist
    save_path = os.path.join(save_dir, 'infection_trend_2010_to_2023.png')
    plt.savefig(save_path)

    # Close the plot to free memory
    plt.close()
    print(f"Combined plot for 2010-2023 saved at {save_path}")

# Assuming df_dict contains all the yearly dataframes, e.g., in_2010, in_2011, ..., in_2023
df_dict = {
    2010: in_2010,
    2011: in_2011,
    2012: in_2012,
    2013: in_2013,
    2014: in_2014,
    2015: in_2015,
    2016: in_2016,
    2017: in_2017,
    2018: in_2018,
    2019: in_2019,
    2020: in_2020,
    2021: in_2021,
    2022: in_2022,
    2023: in_2023
}

# Plot and save the combined trend

plot_combined_infection_trend(df_dict, save_dir)


Combined plot for 2010-2023 saved at D:\Projects\TMU\Indonesia_Dengue\infection_trend_2010_to_2023.png


Visualise on SHP file the areas of interest in a given year.

In [103]:
in_2010

,Province,City,City/Regency,Infection_1,Death_1,Infection_2,Death_2,Infection_3,Death_3,Infection_4,...,Death_10,Infection_11,Death_11,Infection_12,Death_12,Total Incidence,Total Death,IR per 100000,CFR,Population
1,Aceh,Aceh Barat Daya,Regency,0,0,0,0,0,0,0,...,0,0,0,1,0,1.0,0.0,0.870686,0.000000,114852.0
2,Aceh,Aceh Barat,Regency,2,0,5,1,5,0,1,...,0,1,0,0,0,27.0,1.0,17.810731,3.703704,151594.0
3,Aceh,Aceh Besar,Regency,48,0,26,0,37,0,60,...,1,58,0,81,2,539.0,3.0,180.108533,0.556586,299264.0
4,Aceh,Aceh Jaya,Regency,0,0,0,0,1,0,3,...,0,0,0,0,0,6.0,0.0,9.004682,0.000000,66632.0
5,Aceh,Aceh Selatan,Regency,21,0,24,0,13,0,4,...,0,6,0,9,0,108.0,0.0,57.170384,0.000000,188909.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,Yogyakarta,Bantul,Regency,206,0,151,3,152,3,166,...,0,97,0,76,0,1516.0,8.0,182.221394,0.527704,831955.0
441,Yogyakarta,Gunung Kidul,Regency,183,2,204,4,200,1,143,...,0,5,1,2,1,944.0,14.0,131.559876,1.483051,717544.0
442,Yogyakarta,Kulon Progo,Regency,52,0,56,0,77,2,57,...,0,31,0,30,0,467.0,4.0,101.691741,0.856531,459231.0
443,Yogyakarta,Sleman,Regency,148,1,120,1,83,0,71,...,0,21,0,30,0,600.0,2.0,65.09336,0.333333,921753.0
